1. Create notebook = done

In [116]:
import pandas as pd
import numpy as np
from scipy import stats
from pydataset import data
import env
import os
import acquire 
import prepare

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

**2. In the context of this problem, what is a false positive?**

**In the context of this problem, what is a false negative?**

Dog is positive, Cat is negative

FP = actual cat, predicted dog
FN = actual dog, predicted cat

Model is a crosstab of two variables

**3. An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?**

In [5]:
abnormal = pd.read_csv('c3.csv')
abnormal.head()

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect


In [6]:
abnormal.actual.value_counts()

No Defect    184
Defect        16
Name: actual, dtype: int64

In [7]:
abnormal['baseline'] = 'No Defect'
abnormal.head()

,actual,model1,model2,model3,baseline
0,No Defect,No Defect,Defect,No Defect,No Defect
1,No Defect,No Defect,Defect,Defect,No Defect
2,No Defect,No Defect,Defect,No Defect,No Defect
3,No Defect,Defect,Defect,Defect,No Defect
4,No Defect,No Defect,Defect,No Defect,No Defect


In [8]:
abnormal.shape[0]

200

In [9]:
# Percentage of defects for entire batch
percent_of_defects = (abnormal['actual']=='Defect').sum() / abnormal.shape[0]
percent_of_defects

0.08

In [10]:
# Baseline Accuracy
baseline_acc = (abnormal['actual'] == abnormal['baseline']).mean()
baseline_acc

0.92

In [11]:
# Accuracy of model 1
modelyek_acc = (abnormal['actual'] == abnormal['model1']).mean()
modelyek_acc

0.95

In [17]:
# Accuracy of model 2
modeldo_acc = (abnormal['actual'] == abnormal['model2']).mean()
modeldo_acc

0.56

In [18]:
# Accuracy of model 3
modelseh_acc = (abnormal['actual'] == abnormal['model3']).mean()
modelseh_acc

0.555

**Precision**

In [19]:
# Precision  TP / TP+FP

subset_p = abnormal[abnormal['model1']=='Defect']
subset_p.head()


,actual,model1,model2,model3,baseline
3,No Defect,Defect,Defect,Defect,No Defect
30,Defect,Defect,No Defect,Defect,No Defect
62,No Defect,Defect,No Defect,No Defect,No Defect
65,Defect,Defect,Defect,Defect,No Defect
70,Defect,Defect,Defect,Defect,No Defect


In [20]:
# Precision cont.

model_p = (subset_p['model1'] == subset_p['actual']).mean()
model_p

0.8

**Recall**

In [21]:
# Recall     TP / TP+FN
subset_r = abnormal[abnormal['actual']=='Defect']
subset_r.head()



,actual,model1,model2,model3,baseline
13,Defect,No Defect,Defect,Defect,No Defect
30,Defect,Defect,No Defect,Defect,No Defect
65,Defect,Defect,Defect,Defect,No Defect
70,Defect,Defect,Defect,Defect,No Defect
74,Defect,No Defect,No Defect,Defect,No Defect


In [22]:
# Recall cont.

model_recall = (subset_r['model1'] == subset_r['actual']).mean()
model_recall

0.5

In [100]:
(subset_r['model2'] == subset_r['actual']).mean()

0.5625

In [101]:
(subset_r['model3'] == subset_r['actual']).mean()

0.8125

<div class="alert alert-block alert-success">
<b> Recall Evaluation Metric<b>:
<br> - Use this metric to find as many bad ducklings per model. This way even if there are false positives, there will be less bad ducklings missed.

**3. Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?**

<div class="alert alert-block alert-success">
<b> Precision Evaluation Metric<b>:
<br> - Use this metric to predict the amount of good ducklings out there, therefore being able to predict the number vacations that will actually be given. Also, less false positives, which means less free vacations given out. Precision model 1 calculated to have an 0.80 accuracy rate.

In [23]:
paws = pd.read_csv('gives_you_paws.csv')

In [24]:
paws.head()

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog


In [25]:
paws.actual.value_counts()
# more dogs than cats. dogs is the baseline

dog    3254
cat    1746
Name: actual, dtype: int64

In [105]:
# Baseline accuracy
paws['baseline'] = 'dog'
print((paws.actual == paws.baseline).mean())
print((paws.actual == paws.model1).mean())
print((paws.actual == paws.model2).mean())
print((paws.actual == paws.model3).mean())
print((paws.actual == paws.model4).mean())

0.6508
0.8074
0.6304
0.5096
0.7426


In [30]:
# Precision accuracy TP / TP+FP
# Precision subsets for the model 
subset1 = paws[paws['model1']=='dog']
subset1.head()

,actual,model1,model2,model3,model4,baseline
1,dog,dog,cat,cat,dog,dog
3,dog,dog,dog,cat,dog,dog
5,dog,dog,dog,dog,dog,dog
7,cat,dog,cat,cat,dog,dog
8,dog,dog,cat,dog,dog,dog


In [33]:
# get the mean of the model that has been subsetted

model = (subset1.actual == subset1.model1).mean()
model


0.8900238338440586

In [34]:
# Recall accuracy TP / TP+FN
# subsets the actual column to subset for the atual positive cases

subset2 = paws[paws['actual'] == 'dog']

model2 = (subset2.actual == subset2.model1).mean()
model2


0.803318992009834

**4a. In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?**

In [ ]:
baseline = .65
precision = .89
recall = .80

Precision has the highest accuracy with 89%. Baseline is at 65% and recall is at 80%.

**4b. Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recommend?**

In [109]:
# Precision for all models

subset1 = paws[paws['model1']=='dog']
subset2 = paws[paws['model2']=='dog']
subset3 = paws[paws['model3']=='dog']
subset4 = paws[paws['model4']=='dog']


print('PRECISION')
print('\n')
print('MODEL 1')
print((subset1.actual == subset1.model1).mean())
print('------------------\n')
print('MODEL 2')
print((subset2.actual == subset2.model2).mean())
print('------------------\n')
print('MODEL 3')
print((subset3.actual == subset3.model3).mean())
print('------------------\n')
print('MODEL 4')
print((subset4.actual == subset4.model4).mean())


PRECISION


MODEL 1
0.8900238338440586
------------------

MODEL 2
0.8931767337807607
------------------

MODEL 3
0.6598883572567783
------------------

MODEL 4
0.7312485304490948


I would recommend precision as it limits the amount of false positives, which means there would be less cats in the dog albums and there would be few actual dogs that are actually predicted as cats. Model 4

**4c. Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recommend?**

In [45]:
# Baseline for cats
paws['baseline_cats'] = 'cat'
paws.head()

,actual,model1,model2,model3,model4,baseline,baseline_cats
0,cat,cat,dog,cat,dog,dog,cat
1,dog,dog,cat,cat,dog,dog,cat
2,dog,cat,cat,cat,dog,dog,cat
3,dog,dog,dog,cat,dog,dog,cat
4,cat,cat,cat,dog,dog,dog,cat


In [47]:
# accuracy of baseline cats for model 2

(paws.actual == paws.baseline_cats).mean()

0.3492

In [52]:
# Accuracy of each model 
print((paws.model1 == paws.actual).mean())
print((paws.model2 == paws.actual).mean())
print((paws.model3 == paws.actual).mean())
print((paws.model4 == paws.actual).mean())

0.8074
0.6304
0.5096
0.7426


In [55]:
subset3= paws[paws['actual']=='cat']
subset3.head()

,actual,model1,model2,model3,model4,baseline,baseline_cats
0,cat,cat,dog,cat,dog,dog,cat
4,cat,cat,cat,dog,dog,dog,cat
6,cat,cat,cat,cat,dog,dog,cat
7,cat,dog,cat,cat,dog,dog,cat
11,cat,cat,dog,cat,cat,dog,cat


In [110]:
subset4 = paws[paws['model1']=='cat']
subset5 = paws[paws['model2']=='cat']
subset6 = paws[paws['model3']=='cat']
subset7 = paws[paws['model4']=='cat']


print('PRECISION')
print('\n')
print('MODEL 1')
print((subset4.actual == subset4.model1).mean())
print('------------------\n')
print('MODEL 2')
print((subset5.actual == subset5.model2).mean())
print('------------------\n')
print('MODEL 3')
print((subset6.actual == subset6.model3).mean())
print('------------------\n')
print('MODEL 4')
print((subset7.actual == subset7.model4).mean())

PRECISION


MODEL 1
0.6897721764420747
------------------

MODEL 2
0.4841220423412204
------------------

MODEL 3
0.358346709470305
------------------

MODEL 4
0.8072289156626506


In [58]:
# Recall for all models TP/ TP+FN

print('RECALL')
print('\n')
print('MODEL 1')
print((subset3.actual == subset3.model1).mean())
print('------------------\n')
print('MODEL 2')
print((subset3.actual == subset3.model2).mean())
print('------------------\n')
print('MODEL 3')
print((subset3.actual == subset3.model3).mean())
print('------------------\n')
print('MODEL 4')
print((subset3.actual == subset3.model4).mean())

RECALL


MODEL 1
0.8150057273768614
------------------

MODEL 2
0.8906071019473081
------------------

MODEL 3
0.5114547537227949
------------------

MODEL 4
0.34536082474226804


<div class="alert alert-block alert-success">
<b> Recall Evaluation Metric<b>:
<br> - This has the highest accuracy and if they are solely dealing with cat pictures, then it is in their benefit to try to identify as many cats as possible. This would also increase false positives as a result.

**5. Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.**

In [69]:
from sklearn.metrics import accuracy_score

print(accuracy_score(paws.actual, paws.model1, normalize=False))
print(accuracy_score(paws.actual, paws.model2))
print(accuracy_score(paws.actual, paws.model3))
print(accuracy_score(paws.actual, paws.model4, normalize=False))


4037
0.6304
0.5096
3713


In [67]:
paws.shape

(5000, 7)

In [93]:
from sklearn.metrics import precision_score

print(precision_score(paws.actual, paws.model1, pos_label='cat'))
print(precision_score(paws.actual, paws.model2, pos_label='cat'))
print(precision_score(paws.actual, paws.model3, pos_label='cat'))
print(precision_score(paws.actual, paws.model4, pos_label='cat'))

print('\n')
print('PRECISION')
print('\n')
print('MODEL 1')
print((subset4.actual == subset4.model1).mean())
print('------------------\n')
print('MODEL 2')
print((subset5.actual == subset5.model2).mean())
print('------------------\n')
print('MODEL 3')
print((subset6.actual == subset6.model3).mean())
print('------------------\n')
print('MODEL 4')
print((subset7.actual == subset7.model4).mean())


0.6897721764420747
0.4841220423412204
0.358346709470305
0.8072289156626506


PRECISION


MODEL 1
0.6897721764420747
------------------

MODEL 2
0.4841220423412204
------------------

MODEL 3
0.358346709470305
------------------

MODEL 4
0.8072289156626506


In [95]:
from sklearn.metrics import recall_score

print(recall_score(paws.actual, paws.model1, pos_label='cat'))
print(recall_score(paws.actual, paws.model2, pos_label='cat'))
print(recall_score(paws.actual, paws.model3, pos_label='cat'))
print(recall_score(paws.actual, paws.model4, pos_label='cat'))

print('\n')
print('RECALL')
print('\n')
print('MODEL 1')
print((subset3.actual == subset3.model1).mean())
print('------------------\n')
print('MODEL 2')
print((subset3.actual == subset3.model2).mean())
print('------------------\n')
print('MODEL 3')
print((subset3.actual == subset3.model3).mean())
print('------------------\n')
print('MODEL 4')
print((subset3.actual == subset3.model4).mean())

0.8150057273768614
0.8906071019473081
0.5114547537227949
0.34536082474226804


RECALL


MODEL 1
0.8150057273768614
------------------

MODEL 2
0.8906071019473081
------------------

MODEL 3
0.5114547537227949
------------------

MODEL 4
0.34536082474226804


In [114]:
from sklearn.metrics import classification_report

classification_report(paws.actual, paws.model1)




'              precision    recall  f1-score   support\n\n         cat       0.69      0.82      0.75      1746\n         dog       0.89      0.80      0.84      3254\n\n    accuracy                           0.81      5000\n   macro avg       0.79      0.81      0.80      5000\nweighted avg       0.82      0.81      0.81      5000\n'

In [119]:
pd.DataFrame(classification_report(paws.actual, paws.model1, labels=['dog','cat'], output_dict=True))

,dog,cat,accuracy,macro avg,weighted avg
precision,0.890024,0.689772,0.8074,0.789898,0.820096
recall,0.803319,0.815006,0.8074,0.809162,0.807400
f1-score,0.844452,0.747178,0.8074,0.795815,0.810484
support,3254.000000,1746.000000,0.8074,5000.000000,5000.000000
